In [1]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
import numpy as np
from scipy import stats
from sklearn.compose import ColumnTransformer

In [27]:
df = pd.read_csv(r"D:\python\projects\car_price_prediction\Samochody Otomoto1")
df.head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor
0,21900,Toyota,RAV4,2003,218000,1794,Benzyna,125,Manualna,5,Niebieski
1,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor
2,32000,Mercedes-Benz,Klasa A,2013,110000,1595,Benzyna,122,Automatyczna,5,Inny kolor
3,34900,Toyota,AvensisGeneracjaIII (2009-),2012,325000,2231,Diesel,149,Manualna,5,Inny kolor
4,26500,Opel,AstraGeneracjaJ (2009-2019),2010,201000,1686,Diesel,110,Manualna,5,Srebrny


In [28]:
labelencoder = LabelEncoder()

In [29]:
for x in df["przebieg"]:
    if len(x) > 10:
        df = df[df["przebieg"] != x]
        
for x in df["cena"]:
    if len(x) > 10:
        df = df[df["cena"] != x]

In [30]:
for x in df["cena"]:
    try:
        x = float(x)
    except:
        df = df[df["cena"] != x]

In [31]:
df["cena"] = pd.to_numeric(df["cena"])
df["marka label"] = labelencoder.fit_transform(df["marka"])
df["model label"] = labelencoder.fit_transform(df["model"])
df["rocznik"] = pd.to_numeric(df["rocznik"])
df["przebieg"] = pd.to_numeric(df["przebieg"])
df["pojemnosc"] = pd.to_numeric(df["pojemnosc"])
df["rodzaj paliwa label"] = labelencoder.fit_transform(df["rodzaj paliwa"])
df["moc"] = pd.to_numeric(df["moc"])
df["skrzynia biegow label"] = labelencoder.fit_transform(df["skrzynia biegow"])
df["liczba drzwi"] = pd.to_numeric(df["liczba drzwi"])
df["kolor label"] = labelencoder.fit_transform(df["kolor"])

In [32]:
df.head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
0,21900,Toyota,RAV4,2003,218000,1794,Benzyna,125,Manualna,5.0,Niebieski,55,734,0,2,94
2,32000,Mercedes-Benz,Klasa A,2013,110000,1595,Benzyna,122,Automatyczna,5.0,Inny kolor,38,534,0,0,89
3,34900,Toyota,AvensisGeneracjaIII (2009-),2012,325000,2231,Diesel,149,Manualna,5.0,Inny kolor,55,138,3,2,89
4,26500,Opel,AstraGeneracjaJ (2009-2019),2010,201000,1686,Diesel,110,Manualna,5.0,Srebrny,43,128,3,2,108
5,37000,Mercedes-Benz,Klasa C,2015,220000,2143,Diesel,204,Automatyczna,4.0,Inny kolor,38,541,3,0,89


In [33]:
df.shape

(12322, 16)

In [34]:
df = df.drop_duplicates()
df = df.dropna()

In [35]:
df.shape

(8079, 16)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8079 entries, 0 to 12355
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cena                   8079 non-null   int64  
 1   marka                  8079 non-null   object 
 2   model                  8079 non-null   object 
 3   rocznik                8079 non-null   int64  
 4   przebieg               8079 non-null   int64  
 5   pojemnosc              8079 non-null   int64  
 6   rodzaj paliwa          8079 non-null   object 
 7   moc                    8079 non-null   int64  
 8   skrzynia biegow        8079 non-null   object 
 9   liczba drzwi           8079 non-null   float64
 10  kolor                  8079 non-null   object 
 11  marka label            8079 non-null   int32  
 12  model label            8079 non-null   int32  
 13  rodzaj paliwa label    8079 non-null   int32  
 14  skrzynia biegow label  8079 non-null   int32  
 15  kol

In [37]:
def find_outliers_limit(df,col):
    print(col)
    print('-'*50)
    #removing outliers
    q25, q75 = np.percentile(df[col], 25), np.percentile(df[col], 75)
    iqr = q75 - q25
    print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))
    # calculate the outlier cutoff
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    print('Lower:',lower,' Upper:',upper)
    return lower,upper
def remove_outlier(df,col,upper,lower):
    # identify outliers
    outliers = [x for x in df[col] if x  == upper]
    print('Identified outliers: %d' % len(outliers))
    # remove outliers
    outliers_removed = [x for x in df[col] if x >= lower and x <= upper]
    print('Non-outlier observations: %d' % len(outliers_removed))
    final= np.where(df[col]>upper,upper,np.where(df[col]<lower,lower,df[col]))
    return final
outlier_cols=["marka label", "model label", "rodzaj paliwa label", 
        "kolor label", "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label", "cena"]
for col in outlier_cols:
    lower,upper=find_outliers_limit(df,col)
    df[col]=remove_outlier(df,col,upper,lower)

marka label
--------------------------------------------------
Percentiles: 25th=14.000, 75th=46.000, IQR=32.000
Lower: -34.0  Upper: 94.0
Identified outliers: 0
Non-outlier observations: 8079
model label
--------------------------------------------------
Percentiles: 25th=210.500, 75th=789.000, IQR=578.500
Lower: -657.25  Upper: 1656.75
Identified outliers: 0
Non-outlier observations: 8079
rodzaj paliwa label
--------------------------------------------------
Percentiles: 25th=0.000, 75th=3.000, IQR=3.000
Lower: -4.5  Upper: 7.5
Identified outliers: 0
Non-outlier observations: 8079
kolor label
--------------------------------------------------
Percentiles: 25th=42.000, 75th=108.000, IQR=66.000
Lower: -57.0  Upper: 207.0
Identified outliers: 0
Non-outlier observations: 8079
rocznik
--------------------------------------------------
Percentiles: 25th=2007.000, 75th=2013.000, IQR=6.000
Lower: 1998.0  Upper: 2022.0
Identified outliers: 0
Non-outlier observations: 8018
przebieg
-----------

In [38]:
data = df[["marka label", "model label", "rodzaj paliwa label", 
        "kolor label", "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label", "cena"]]
X = data[["marka label", "model label", "rodzaj paliwa label", 
        "kolor label", "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]]
y = data["cena"]

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 10)

regressor.fit(X_train, y_train)

[22:43:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10, n_jobs=8,
             num_parallel_tree=1, objective='reg:linear', predictor='auto',
             random_state=0, reg_alpha=10, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [40]:
Y_pred_train = regressor.predict(X_train)
y_pred = regressor.predict(X_test)

In [41]:
def rmsle(y_pred,y_test) :
    error = np.square(np.log10(y_pred +1) - np.log10(y_test +1)).mean() ** 0.5
    Acc = 1 - error
    return Acc

print("Accuracy attained on Training Set = ",rmsle(Y_pred_train, y_train))
print("Accuracy attained on Test Set = ",rmsle(y_pred,y_test))

Accuracy attained on Training Set =  0.7986940971821364
Accuracy attained on Test Set =  0.78922315676022


In [42]:
lr = LinearRegression()
lr.fit(X,y)
lr.score(X,y)

0.7357999908897063

In [43]:
df[df["model"] == "V50"].head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
215,29900.0,Volvo,V50,2012.0,198426.0,1560.0,Diesel,115.0,Manualna,5.0,Czarny,58.0,941.0,3.0,2.0,42.0
528,10899.0,Volvo,V50,2004.0,239000.0,1998.0,Diesel,136.0,Manualna,5.0,Zielony,58.0,941.0,3.0,2.0,143.0
547,13999.0,Volvo,V50,2005.0,225000.0,1998.0,Diesel,136.0,Manualna,5.0,Niebieski,58.0,941.0,3.0,2.0,94.0
819,17900.0,Volvo,V50,2007.0,267000.0,1798.0,Benzyna,125.0,Manualna,5.0,Szary,58.0,941.0,0.0,2.0,119.0
1269,12800.0,Volvo,V50,2005.0,320000.0,1998.0,Diesel,136.0,Manualna,5.0,Czarny,58.0,941.0,3.0,2.0,42.0


In [44]:
["marka label", "model label", "rodzaj paliwa label", "kolor label", "rocznik", 
 "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]
volvo = [45, 575, 2, 20, 2005, 260333, 1998.0, 170, 5, 1]
volvo = np.array(volvo).reshape((1,-1))
regressor.predict(volvo)

array([17629.447], dtype=float32)

In [45]:
df[df["model"] == "SX4"].head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
3495,22900.0,Suzuki,SX4,2007.0,188355.0,1586.0,Benzyna,107.0,Manualna,5.0,Czarny,53.0,800.0,0.0,2.0,42.0
4237,31900.0,Suzuki,SX4,2012.0,143000.0,1586.0,Benzyna,120.0,Manualna,5.0,Szary,53.0,800.0,0.0,2.0,119.0
4466,19900.0,Suzuki,SX4,2010.0,131551.0,1586.0,Benzyna,120.0,Manualna,5.0,Czarny,53.0,800.0,0.0,2.0,42.0
5115,13500.0,Suzuki,SX4,2006.0,257000.0,1586.0,Benzyna,107.0,Manualna,5.0,Czerwony,53.0,800.0,0.0,2.0,74.0
5139,22900.0,Suzuki,SX4,2009.0,199245.0,1586.0,Benzyna,107.0,Automatyczna,5.0,Czerwony,53.0,800.0,0.0,0.0,74.0


In [46]:
["marka label", "model label", "rodzaj paliwa label", "kolor label", 
 "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]
suzuki = [42, 485, 0, 37, 2007, 10700, 1586.0, 100, 5, 1]
suzuki = np.array(suzuki).reshape((1,-1))
regressor.predict(suzuki)

array([17384.615], dtype=float32)

In [47]:
df[df["model"] == "Fabia"].head()

,cena,marka,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
107,33333.0,Škoda,Fabia,2015.0,27000.0,999.0,Benzyna,60.0,Manualna,5.0,Inny kolor,59.0,349.0,0.0,2.0,89.0
109,34700.0,Škoda,Fabia,2015.0,65000.0,999.0,Benzyna,60.0,Manualna,5.0,Niebieski,59.0,349.0,0.0,2.0,94.0
237,21900.0,Škoda,Fabia,2014.0,224257.0,1598.0,Diesel,90.0,Manualna,5.0,Srebrny,59.0,349.0,3.0,2.0,108.0
317,28900.0,Škoda,Fabia,2013.0,179000.0,1598.0,Diesel,105.0,Manualna,5.0,Czerwony,59.0,349.0,3.0,2.0,74.0
566,3900.0,Škoda,Fabia,2001.0,197789.0,1390.0,Benzyna,75.0,Manualna,5.0,Czarny,59.0,349.0,0.0,2.0,42.0


In [48]:
["marka label", "model label", "rodzaj paliwa label", "kolor label", 
 "rocznik", "przebieg", "pojemnosc", "moc", "liczba drzwi", "skrzynia biegow label"]
skoda = [46, 218, 2, 20, 2013, 185000, 1598.0, 100, 5, 1]
skoda = np.array(skoda).reshape((1,-1))
regressor.predict(skoda)

array([24220.834], dtype=float32)

In [52]:
df.groupby("marka").count().sort_values("cena").tail()

,cena,model,rocznik,przebieg,pojemnosc,rodzaj paliwa,moc,skrzynia biegow,liczba drzwi,kolor,marka label,model label,rodzaj paliwa label,skrzynia biegow label,kolor label
marka,,,,,,,,,,,,,,,
Volkswagen,592,592,592,592,592,592,592,592,592,592,592,592,592,592,592
Ford,618,618,618,618,618,618,618,618,618,618,618,618,618,618,618
Audi,744,744,744,744,744,744,744,744,744,744,744,744,744,744,744
Opel,762,762,762,762,762,762,762,762,762,762,762,762,762,762,762
BMW,773,773,773,773,773,773,773,773,773,773,773,773,773,773,773
